### Test: 空頭大盤破底期間，回檔較小、甚至上漲的族群，是否會是反彈後的強勢族群

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import numpy as np
import warnings
warnings.filterwarnings('ignore')

stock_price = pd.read_csv("Stock_Price_Feb02_2024")
stock_price["Ticker"] = stock_price["Ticker"].astype(str)
stock_price["Date"] = pd.to_datetime(stock_price["Date"], format='%Y-%m-%d')

time_series = stock_price["Date"]
formatted_dates = time_series.dt.strftime('%Y-%m-%d')
date_list = list(formatted_dates)

In [2]:
def rs_calculation(start_date, end_date):
    filter1 = stock_price[stock_price["Date"] == start_date]; list1 = list(filter1["Ticker"])
    filter2 = stock_price[stock_price["Date"] == end_date]; list2 = list(filter2["Ticker"])
    ticker_list = [tickers for tickers in list1 if tickers in list2]
    
    per = []
    price_data = stock_price[(stock_price["Ticker"].isin(ticker_list)) & (stock_price["Date"].isin([start_date, end_date]))]
    for ticker in ticker_list:
        ticker_data = price_data[price_data["Ticker"] == ticker]
        start_price = ticker_data[ticker_data["Date"] == start_date]["Close"].values[0]
        end_price = ticker_data[ticker_data["Date"] == end_date]["Close"].values[0]
        per.append(round((end_price - start_price) / start_price, 5))
    
    rs_data = [round(((sorted(per).index(x) + 1) / len(per) * 100), 2) for x in per]
    date = [end_date]*len(rs_data)
    df = pd.DataFrame({
        "Date": date,
        "Ticker": ticker_list,
        "RS": rs_data
    })
    return df

In [3]:
group = {"重電": ["1519", "1513", "1514", "1503"], # 華城、中興電、亞力、士電
         "軸承": ["3548", "6805", "3376"], # 兆利、富世達、新日興 
         "IP": ["3661", "3443", "3529", "3035", "6643"], # 世芯、創意、力旺、智原、M31
         "網通": ["6285", "3596", "5388", "4906", "3694", "2345", "6546"], # 啟碁、智易、中磊、正文、海華、智邦、正基
         "散熱": ["3017", "3324", "3653", "2421"], # 奇鋐、雙鴻、健策、建準
         "機殼": ["8210", "3325", "5426", "3693", "3013"], # 勤誠、旭品、振發、營邦、晟銘電
         "伺服器": ["2382", "3231", "6669", "2376", "2356"], # 廣達、緯創、緯穎、技嘉、英業達
         "PC": ["2353", "2357", "2324"], # 宏碁、華碩、仁寶
         "記憶體": ["8299", "3260", "2408", "2344", "8150"], # 群聯、威剛、南亞科、華邦電、南茂
         "IC設計": ["2454", "2379", "3034", "4966"], # 聯發科、瑞昱、聯詠、譜瑞-ky
         "IC代工": ["2330", "2303", "5347"], # 台積電、聯電、世界先進
         "IC封測": ["3711", "6239"], # 日月光、力成
         "IC檢測": ["3289", "3587"], # 宜特、閎康
         "矽晶圓": ["6488", "6182"], # 環球晶、合晶
         "再生晶圓": ["1560", "3583", "8028"], # 中砂、辛耘、昇陽半
         "PCB": ["2313", "2368", "4958"], # 華通、金像電、臻鼎
         "光學鏡頭": ["3008", "3406", "3504", "3362", "3019"], # 大立光、玉晶光、揚明光、先進光、亞光
         "PMIC": ["6138", "8081", "6719", "6415", "6799"], # 茂達、致新、力智、矽力-ky、來頡
         "銅箔基板": ["2383", "6274", "6213", "8358"], # 台光電、台燿、聯茂、金居
         "被動元件": ["2327", "2492", "3026"], # 國巨、華新科、禾伸堂
         "光通訊": ["3081", "6426", "3234", "4979", "6451", "3450", "8011", "6442", "4908", "4977", "3163", "6530"],
                   # 聯亞、統新、光環、華星光、訊芯-ky、聯均、上詮、台通、光聖、前鼎、眾達-ky、波若威、創威
         "工業電腦": ["2395", "6414", "3022", "6579"], # 研華、樺漢、威強電、研揚
         "電子支付": ["6206", "8114", "2482", "5258"], # 飛捷、振樺電、連宇、虹堡
         "水資源": ["8936", "8473"], # 國統、山林水
         "車用PCB": ["2355", "3044", "3715", "2367"], # 敬鵬、健鼎、定穎投控、燿華
         "車用零組件": ["4721", "4739", "6271", "8255"], # 美琪瑪、康普、同欣電、朋程
         "車用AM": ["1319", "1522", "1524", "6605"], # 東陽、堤維西、耿鼎、帝寶
         "軍工": ["2634", "8222", "8033", "2630", "1584", "6829"], # 漢翔、寶一、雷虎、亞航、精剛、千附精密
         "租賃": ["5871", "9941", "6592"], # 中租、裕融、和潤 
         "自行車": ["9914", "9921"], # 美利達、巨大
         "面板": ["2409", "3481"], # 友達、群創
         "製衣": ["1477", "1476"], # 聚陽、儒鴻
         "製鞋": ["9904", "9910", "9802"], # 寶成、豐泰、鈺齊-ky
         "神盾集團": ["6462", "8054", "6684", "6237", "6243", "6695"], # 神盾、安國、安格、驊訊、迅杰、芯鼎
         "元宇宙": ["2498", "6118", "2388", "8450"], # 宏達電、建達、威盛、霹靂、位速
         "航空": ["2618", "2610"], # 長榮航、華航
         "海運": ["2618", "2609", "2615"]} # 長榮、陽明、萬海

group_keys = list(group.keys())
group_values = list(group.values())

#### 空頭期1: 2023-12-29 ~ 2024-01-17, 反彈期1: 2024-01-18 ~ 2024-01-29

In [5]:
correction = rs_calculation("2023-12-29", "2024-01-17")
correction_1 = rs_calculation("2023-12-29", "2024-01-09")
correction_2 = rs_calculation("2024-01-09", "2024-01-17")
rebound = rs_calculation("2024-01-17", "2024-02-02")
rebound_1 = rs_calculation("2024-01-17", "2024-01-29")
rebound_2 = rs_calculation("2024-01-29", "2024-02-02")

correction_rs = []; correction_1_rs = []; correction_2_rs = []
rebound_rs = []; rebound_1_rs = []; rebound_2_rs = []
for i in range(len(group_keys)):
    correction_total = 0; correction_1_total = 0; correction_2_total = 0
    rebound_total = 0; rebound_1_total = 0; rebound_2_total = 0
    for j in range(len(group_values[i])):
        rs = correction[correction["Ticker"] == group_values[i][j]]["RS"].values[0]; correction_total = correction_total + rs
        rs = correction_1[correction_1["Ticker"] == group_values[i][j]]["RS"].values[0]; correction_1_total = correction_1_total + rs
        rs = correction_2[correction_2["Ticker"] == group_values[i][j]]["RS"].values[0]; correction_2_total = correction_2_total + rs
        rs = rebound[rebound["Ticker"] == group_values[i][j]]["RS"].values[0]; rebound_total = rebound_total + rs
        rs = rebound_1[rebound_1["Ticker"] == group_values[i][j]]["RS"].values[0]; rebound_1_total = rebound_1_total + rs
        rs = rebound_2[rebound_2["Ticker"] == group_values[i][j]]["RS"].values[0]; rebound_2_total = rebound_2_total + rs
    correction_rs.append(round(correction_total / len(group_values[i]), 2))
    correction_1_rs.append(round(correction_1_total / len(group_values[i]), 2))
    correction_2_rs.append(round(correction_2_total / len(group_values[i]), 2))
    rebound_rs.append(round(rebound_total / len(group_values[i]), 2))
    rebound_1_rs.append(round(rebound_1_total / len(group_values[i]), 2))
    rebound_2_rs.append(round(rebound_2_total / len(group_values[i]), 2))
    
df = {"族群": group_keys,
      "總回檔": correction_rs,
      "第一波回檔": correction_1_rs,
      "第二波回檔": correction_2_rs,
      "總反彈": rebound_rs,
      "第一波反彈": rebound_1_rs,
      "第二波反彈": rebound_2_rs
}

df = pd.DataFrame(df).sort_values(by="第二波回檔", ascending=False).reset_index(drop=True)
df.head()

,族群,總回檔,第一波回檔,第二波回檔,總反彈,第一波反彈,第二波反彈
0,神盾集團,90.14,67.35,95.74,90.36,49.98,85.99
1,IC檢測,92.79,69.31,94.18,61.07,36.92,87.40
2,軸承,92.56,68.93,91.61,95.85,34.63,98.82
3,散熱,45.91,3.82,89.31,97.02,73.97,98.03
4,伺服器,63.36,15.19,81.87,95.42,95.50,87.82


In [7]:
df = pd.DataFrame(df).sort_values(by="總反彈", ascending=False).reset_index(drop=True)
df.head()

,族群,總回檔,第一波回檔,第二波回檔,總反彈,第一波反彈,第二波反彈
0,散熱,45.91,3.82,89.31,97.02,73.97,98.03
1,軸承,92.56,68.93,91.61,95.85,34.63,98.82
2,伺服器,63.36,15.19,81.87,95.42,95.50,87.82
3,水資源,62.83,59.16,62.62,91.40,67.69,93.09
4,神盾集團,90.14,67.35,95.74,90.36,49.98,85.99


##### 可以看出大盤第二波回檔時走勢較強的族群 ，在大盤反彈後極有可能是下一個主流族群